In [1]:
import streamlit as st
import google.generativeai as genai
import os
import PyPDF2 as pdf
from dotenv import load_dotenv
from langchain import PromptTemplate
from google.cloud import aiplatform
from google.cloud.aiplatform.v1 import schema
import json


In [2]:

from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
KEY = os.getenv("GOOGLE_GEMINI_API_KEY")


In [4]:
from google.cloud import aiplatform
# from google.cloud.aiplatform.gapic.schema import predict
# from google.cloud.aiplatform.gapic.schema import predict_response

# Assuming you have already set the environment variable KEY with your API key

# Create an instance of the ChatGoogleGemini class
model = aiplatform.gapic.models.ModelServiceClient().model(name="projects/YOUR_PROJECT_ID/models/gemini-pro")



AttributeError: module 'google.cloud.aiplatform.gapic' has no attribute 'models'

In [ ]:

#model=genai.GenerativeModel('gemini-pro')

llm = ChatGoogleGemini(api_key=KEY, model_name="gemini-pro", temperature=0.5)



In [ ]:

RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain],
                                          input_variables=["text", "number", "subject", "tone", "response_json"],
                                          output_variables=["quiz", "review"], verbose=True,)
file_path = r"C:\Users\sunny\mcqgen\data.txt"
file_path
with open(file_path, 'r') as file:
    TEXT = file.read()
print(TEXT)
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)
NUMBER = 5
SUBJECT = "biology"
TONE = "simple"

# How to setup Token Usage Tracking in LangChain
with get_google_gemini_callback() as cb:  # Assuming the existence of a Google Gemini callback
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")
response
quiz = response.get("quiz")
quiz = json.loads(quiz)
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})
quiz_table_data
quiz = pd.DataFrame(quiz_table_data)
quiz.to_csv("machinelearning.csv", index=False)
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')
